In [1]:
# import the necessary packages
import imutils
import cv2
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from skimage.feature import hog

In [2]:
#                           START READING FILES FOR VEHICLES AND NON-VEHICLES
# Reading car images
vehicle_image_arr1 = glob.glob('/media/rd_square/Important/Image_processing/car_data_with_neg/vehicles/*/*.png')

# read images, flip images and append into list
vehicle_images_list = []    
for imagePath in vehicle_image_arr1:
    readImage = cv2.imread(imagePath)
    rgbImage = cv2.cvtColor(readImage, cv2.COLOR_BGR2RGB)
    rgbImage = cv2.resize(rgbImage, (64,64))
    flippedImage = cv2.flip(rgbImage, flipCode=1) # flip horizontally
    vehicle_images_list.append(rgbImage)
    vehicle_images_list.append(flippedImage)
    
print("Reading of vehicle images done...")

Reading of vehicle images done...


In [3]:

# Reading non car images
no_vehicle_image_arr1 = glob.glob('/media/rd_square/Important/Image_processing/car_data_with_neg/non-vehicles/*/*.png')
no_vehicle_image_arr2 = glob.glob('/media/rd_square/Important/Image_processing/haar_cascade_classifier/neg/*.jpg')

# read images, flip images and append into list
no_vehicle_images_list = []
for imagePath in no_vehicle_image_arr1:
    readImage = cv2.imread(imagePath)
    rgbImage = cv2.cvtColor(readImage, cv2.COLOR_BGR2RGB)
    rgbImage = cv2.resize(rgbImage, (64,64))
    flippedImage = cv2.flip(rgbImage, flipCode=1) # flip horizontally
    no_vehicle_images_list.append(rgbImage)
    no_vehicle_images_list.append(flippedImage)
    
for imagePath in no_vehicle_image_arr2:
    readImage = cv2.imread(imagePath)
    rgbImage = cv2.cvtColor(readImage, cv2.COLOR_BGR2RGB)
    rgbImage = cv2.resize(rgbImage, (64,64))
    no_vehicle_images_list.append(rgbImage)
    
    
print("Reading of non vehicle images done...")

Reading of non vehicle images done...


In [4]:

print("No of Vehicles images: ", len(vehicle_images_list))
print("No of non vehicle images: ", len(no_vehicle_images_list))
#                       ENDING OF READING IMAGES FROM FILES


No of Vehicles images:  1150
No of non vehicle images:  1038


In [5]:

#                       FEATURE EXTRACTION FROM IMAGES

def GetFeaturesFromHog(image, orient, cellsPerBlock, pixelsPerCell,
                       visualise=False, feature_vector_flag=True):
    'To extract hog feature using hog() from YUV image'
    hog_features = hog(image, orientations=orient,
                                  pixels_per_cell=(pixelsPerCell, pixelsPerCell),
                                  cells_per_block=(cellsPerBlock, cellsPerBlock),
                                  visualize=visualise, feature_vector= feature_vector_flag, block_norm='L2-Hys')
    return hog_features

def ExtractFeatures(images, orientation=9, cellsPerBlock=2, pixelsPerCell=16, convertColorSpace=True):
    'Arrangin features into horizontal stack extracted from hog()'
    featureList = []
    for image in images:
        if(convertColorSpace):
            image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        feature1 = GetFeaturesFromHog(image[:,:,0], orientation, cellsPerBlock, pixelsPerCell)
        feature2 = GetFeaturesFromHog(image[:,:,1], orientation, cellsPerBlock, pixelsPerCell)
        feature3 = GetFeaturesFromHog(image[:,:,2], orientation, cellsPerBlock, pixelsPerCell)

        # Arranging feature into horizontal stack for machine learning model training set
        featureListing = np.hstack((feature1, feature2, feature3))
        featureList.append(featureListing)
    return featureList

In [6]:
vehicleFeatures = ExtractFeatures(vehicle_images_list)
nonVehicleFeatures = ExtractFeatures(no_vehicle_images_list)

''' Arranging features into vertical stack for training the machine learning model '''
featureList = np.vstack([vehicleFeatures, nonVehicleFeatures])
print("Shape of features list is ", featureList.shape)
labelList = np.concatenate([np.ones(len(vehicleFeatures)), np.zeros(len(nonVehicleFeatures))])
print("Shape of labels list is ", labelList.shape)

Shape of features list is  (2188, 972)
Shape of labels list is  (2188,)


In [7]:
#                       DATA PREPROCESSING

# train and test split of data
X_train, X_test, Y_train, Y_test = train_test_split(featureList, labelList, test_size=0.2, shuffle=True)


#                       TRAINING THE LINEAR SUPPORT VECTOR CLASSIFIER MODEL OF MACHINE LEARNING

svcClassifier  = LinearSVC()
svcClassifier.fit(X_train, Y_train) # training the classifier
print("Support Vector Classifier Trained...")
print("Accuracy of trained svm classifier is ", svcClassifier.score(X_test, Y_test))

Support Vector Classifier Trained...
Accuracy of trained svm classifier is  0.9908675799086758


In [13]:

video = cv2.VideoCapture('camera.mp4') # Capturing video from camera

# Initialize the first frame in the video stream
firstFrame = None
count = 0

# loop over the frames of the video
while True:
    ret, frame = video.read()

    if frame is None:
        break


    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (31,31),0)

    # if the first frame is None, initialize it
    if count < 2:
        firstFrame = gray
        count += 1
        continue

    # Compute the absolute difference between the current frame and first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255,cv2.THRESH_BINARY)[1] # use inverse binary threshold when needed

    # dilate the thresholded image to fill in holes,
    # then find contours on thresholded image
    #thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    #thresh = cv2.dilate(thresh, cv2.getStructuringElement(), iterations=2)
    
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # loop over the contours
    for c in cnts:
        if cv2.contourArea(c) < 200:
            continue
        (x,y,w,h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x,y), (x+w,y+h),(0,255,0),2)
    
    cv2.imshow('frame1', frame)
    cv2.imshow('threshold', thresh)
    k = cv2.waitKey(0)
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()